In [1]:
import sys
from collections import namedtuple
from os import listdir, makedirs
from os.path import isfile, isdir, join, exists, abspath
import random
import math
import numpy as np
import trimesh as tri
import trimesh_obb
import createViews
import evaluate_sample
import operator
import pickle
import kmedian as km
import pymesh

# Mix-n-Match Modelling

## Approach

### Loading

In [2]:
Template = namedtuple("Template", "dir templateParts")
TemplatePart = namedtuple("TemplatePart", "dir obj boundingBox orientedExtents symmetries")

In [3]:
def loadTemplatesWithoutPickle():
    templates = []
    parts = []

    # Read all parts for all chairs and create the templates
    chairDirs = [f for f in listdir(chairsDirectory) if isdir(join(chairsDirectory, f))]
    for chairDir in chairDirs:
        print("Loading tempate from: " + chairDir)
        newTemplate = Template(chairDir, [])
        chairMeshes = chairsDirectory + "/" + chairDir + "/meshes"
        chairParts = [f for f in listdir(chairMeshes) if isfile(join(chairMeshes, f))]
        for chairPart in chairParts:
            mesh = pymesh.load_mesh(chairMeshes + "/" + chairPart)
            boundingBox = computeOBB(mesh)
            orientedExtents = tri.bounds.oriented_bounds(boundingBox)[1]
            newTemplatePart = TemplatePart(chairDir, chairPart, boundingBox, orientedExtents, [])
            newTemplate.templateParts.append(newTemplatePart)
            parts.append(newTemplatePart)

        # Find symmetries
        for templatePart1 in newTemplate.templateParts:
            for templatePart2 in newTemplate.templateParts:
                if templatePart1 != templatePart2:
                    symmetry = checkForSymmetry(templatePart1, templatePart2)
                    if symmetry[0]:
                        print("\tX Symmetry Detected: " + templatePart1.obj + " ; " + templatePart2.obj)
                    if symmetry[1]:
                        print("\tY Symmetry Detected: " + templatePart1.obj + " ; " + templatePart2.obj)
                    if symmetry[2]:
                        print("\tZ Symmetry Detected: " + templatePart1.obj + " ; " + templatePart2.obj)
                    if any(axis == True for axis in symmetry):
                        templatePart1.symmetries.append([templatePart2, symmetry])

        templates.append(newTemplate)

    return [templates, parts]

def loadTemplates():
    templates = []
    parts = []

    try:
        with open("templates", 'rb') as f:
            templates, parts = pickle.load(f)
            print('Sucessfully found templates file')
            return [templates, parts]
    except:
        templates, parts = loadTemplatesWithoutPickle()
        with open('templates', 'wb') as f:
            pickle.dump([templates, parts], f)
        return [templates, parts]



### Create OBB

In [4]:
def computeOBB(pymesh):
    trimesh = trimesh_obb.convertPymeshToTrimesh(pymesh);
    boundingBox = trimesh_obb.convertMeshToObb(trimesh)
    return boundingBox

### Deformation Cost

In [5]:
def calculateDeformationCostProcrustes(target, candidate):
    t, _, c = trimesh_obb.procrustesMatrixCost(candidate.boundingBox, target.boundingBox)
    return t, c

def calculateDeformationCost(target, candidate):
    centroidDifference = target.boundingBox.centroid - candidate.boundingBox.centroid

    tarExtents = target.orientedExtents
    canExtents = candidate.orientedExtents
    extentsDifference = tarExtents - canExtents

    rotationTarget = np.delete(np.delete(target.boundingBox.bounding_box_oriented.primitive.transform, 3, 0), 3, 1)
    rotationCandidate = np.delete(np.delete(candidate.boundingBox.bounding_box_oriented.primitive.transform, 3, 0), 3, 1)
    rotationDifference = np.matmul(rotationCandidate, np.transpose(rotationTarget))
    rotationDifferenceTrace = np.trace(rotationDifference)
    if rotationDifferenceTrace > 3.0 :
        rotationDifferenceTrace = 3.0
    if rotationDifferenceTrace < -1.0 :
        rotationDifferenceTrace = -1.0
    rotationDistance = math.acos((rotationDifferenceTrace - 1.0) / 2.0)

    # Need to come up with a better function or a better way to tune the weights
    weights = [0.3, 0.3, 0.0, 0.4]
    t, c = calculateDeformationCostProcrustes(target, candidate)
    return np.sum(weights[0] * np.square(centroidDifference) + weights[1] * np.square(extentsDifference) + weights[2] * np.square(rotationDistance) + weights[3] * c)

### Connect mesh

In [6]:
def matchOBB(target, candidate, mesh):
    # Transforms the candidate mesh so its obb matches the obb of the target
    # Currently seems like only translation works well

    tarCent = target.boundingBox.centroid
    canCent = candidate.boundingBox.centroid
    tarExtents = target.orientedExtents
    canExtents = candidate.orientedExtents

    scaleExtents = tarExtents / canExtents;

    transform = np.array([[1,0,0,0],
        [0,1,0,0],
        [0,0,1,0],
        [0,0,0,1]])

    canToOrg = np.array([[1,0,0,-canCent[0]],
              [0,1,0,-canCent[1]],
              [0,0,1,-canCent[2]],
              [0,0,0,1]])

    rotationCandidate = np.delete(np.delete(candidate.boundingBox.bounding_box_oriented.primitive.transform, 3, 0), 3, 1)
    canRotInv = np.array([[rotationCandidate[0][0],rotationCandidate[0][1],rotationCandidate[0][2],0],
              [rotationCandidate[1][0],rotationCandidate[1][1],rotationCandidate[1][2],0],
              [rotationCandidate[2][0],rotationCandidate[2][1],rotationCandidate[2][2],0],
              [0,0,0,1]])
    canRotInv = np.transpose(canRotInv)

    scale = np.array([[scaleExtents[0],0,0,0],
              [0,scaleExtents[1],0,0],
              [0,0,scaleExtents[2],0],
              [0,0,0,1]])

    rotationTarget = np.delete(np.delete(target.boundingBox.bounding_box_oriented.primitive.transform, 3, 0), 3, 1)
    tarRot = np.array([[rotationTarget[0][0],rotationTarget[0][1],rotationTarget[0][2],0],
              [rotationTarget[1][0],rotationTarget[1][1],rotationTarget[1][2],0],
              [rotationTarget[2][0],rotationTarget[2][1],rotationTarget[2][2],0],
              [0,0,0,1]])

    orgToTar = np.array([[1,0,0,tarCent[0]],
              [0,1,0,tarCent[1]],
              [0,0,1,tarCent[2]],
              [0,0,0,1]])

    transform = np.matmul(transform, orgToTar)
    transform = np.matmul(transform, tarRot)
    transform = np.matmul(transform, scale)
    transform = np.matmul(transform, canRotInv)
    transform = np.matmul(transform, canToOrg)

    newVertices = np.ndarray(mesh.vertices.shape, mesh.vertices.dtype)
    for index, vertex in enumerate(mesh.vertices):

        homVertex = np.array([vertex[0], vertex[1], vertex[2], 1])
        transformedVertex = np.matmul(transform, homVertex)
        newVertices[index] = np.array([transformedVertex[0], transformedVertex[1], transformedVertex[2]]) / transformedVertex[3]

    return pymesh.form_mesh(newVertices, mesh.faces)

In [7]:
def addToNewMesh(newMesh, selectedMesh):
    if newMesh == None:
        newMesh = selectedMesh
    else :
        # selectedMesh = connectPartToMesh(newMesh, selectedMesh)
        newMesh = pymesh.merge_meshes([newMesh, selectedMesh])
    return newMesh

def addToObbMesh(obbMesh, templatePart):
    boundingBox = templatePart.boundingBox
    selectedObb = trimesh_obb.convertObbToMesh(boundingBox);
    if obbMesh == None:
        obbMesh = selectedObb
    else :
        obbMesh = pymesh.merge_meshes([obbMesh, selectedObb])
    return obbMesh

### Symmetry

In [8]:
def checkForSymmetry(part1, part2):
    # X axis symmetry
    part1Copy = part1.boundingBox.copy()
    part2Copy = part2.boundingBox.copy()

    reflectX = np.array([
        [-1,0,0,0],
        [0,1,0,0],
        [0,0,1,0],
        [0,0,0,1]])

    part1Copy.apply_transform(reflectX)

    center1 = np.array([
        [1,0,0,-part1Copy.centroid[0]],
        [0,1,0,0],
        [0,0,1,0],
        [0,0,0,1]])

    center2 = np.array([
        [1,0,0,-part2Copy.centroid[0]],
        [0,1,0,0],
        [0,0,1,0],
        [0,0,0,1]])

    part1Copy.apply_transform(center1)
    part2Copy.apply_transform(center2)

    closestPointsX = tri.proximity.ProximityQuery(part1Copy).vertex(part2Copy.vertices)
    differenceX = np.sum(closestPointsX[0])

    # Y axis symmetry
    part1Copy = part1.boundingBox.copy()
    part2Copy = part2.boundingBox.copy()

    reflectY = np.array([
        [1,0,0,0],
        [0,-1,0,0],
        [0,0,1,0],
        [0,0,0,1]])

    part1Copy.apply_transform(reflectY)

    center1 = np.array([
        [1,0,0,0],
        [0,1,0,-part1Copy.centroid[1]],
        [0,0,1,0],
        [0,0,0,1]])

    center2 = np.array([
        [1,0,0,0],
        [0,1,0,-part2Copy.centroid[1]],
        [0,0,1,0],
        [0,0,0,1]])

    part1Copy.apply_transform(center1)
    part2Copy.apply_transform(center2)

    closestPointsY = tri.proximity.ProximityQuery(part1Copy).vertex(part2Copy.vertices)
    differenceY = np.sum(closestPointsY[0])

    # Z axis symmetry
    part1Copy = part1.boundingBox.copy()
    part2Copy = part2.boundingBox.copy()

    reflectZ = np.array([
        [1,0,0,0],
        [0,1,0,0],
        [0,0,-1,0],
        [0,0,0,1]])

    part1Copy.apply_transform(reflectZ)

    center1 = np.array([
        [1,0,0,0],
        [0,1,0,0],
        [0,0,1,-part1Copy.centroid[2]],
        [0,0,0,1]])

    center2 = np.array([
        [1,0,0,0],
        [0,1,0,0],
        [0,0,1,-part2Copy.centroid[2]],
        [0,0,0,1]])

    part1Copy.apply_transform(center1)
    part2Copy.apply_transform(center2)

    closestPointsZ = tri.proximity.ProximityQuery(part1Copy).vertex(part2Copy.vertices)
    differenceZ = np.sum(closestPointsZ[0])

    # Return symmetries
    symmetryThreshold = 0.1
    symmetry = [False, False, False]
    if differenceX < symmetryThreshold:
        symmetry[0] = True
    if differenceY < symmetryThreshold:
        symmetry[1] = True
    if differenceZ < symmetryThreshold:
        symmetry[2] = True
    return symmetry

In [9]:
def getSymmetric(templatePart, symmetryPart, symmetryAxes, selectedMesh):
    canCent = templatePart.boundingBox.centroid
    tarCent = symmetryPart.boundingBox.centroid

    transform = np.array([[1,0,0,0],
        [0,1,0,0],
        [0,0,1,0],
        [0,0,0,1]])

    canToOrg = np.array([[1,0,0,-canCent[0]],
        [0,1,0,-canCent[1]],
        [0,0,1,-canCent[2]],
        [0,0,0,1]])

    reflectX = np.array([
        [-1,0,0,0],
        [0,1,0,0],
        [0,0,1,0],
        [0,0,0,1]])
    reflectY = np.array([
        [1,0,0,0],
        [0,-1,0,0],
        [0,0,1,0],
        [0,0,0,1]])
    reflectZ = np.array([
        [1,0,0,0],
        [0,1,0,0],
        [0,0,-1,0],
        [0,0,0,1]])

    orgToTar = np.array([[1,0,0,tarCent[0]],
        [0,1,0,tarCent[1]],
        [0,0,1,tarCent[2]],
        [0,0,0,1]])

    transform = np.matmul(transform, orgToTar)
    if symmetryAxes[0]:
        transform = np.matmul(transform, reflectX)
    elif symmetryAxes[1]:
        transform = np.matmul(transform, reflectY)
    elif symmetryAxes[2]:
        transform = np.matmul(transform, reflectZ)
    transform = np.matmul(transform, canToOrg)

    newVertices = np.ndarray(selectedMesh.vertices.shape, selectedMesh.vertices.dtype)
    for index, vertex in enumerate(selectedMesh.vertices):

        homVertex = np.array([vertex[0], vertex[1], vertex[2], 1])
        transformedVertex = np.matmul(transform, homVertex)
        newVertices[index] = np.array([transformedVertex[0], transformedVertex[1], transformedVertex[2]]) / transformedVertex[3]

    newFaces = selectedMesh.faces[:,::-1]
    symmetricMesh = pymesh.form_mesh(newVertices, newFaces)
    return symmetricMesh

### Generate new chair from template

In [10]:
def generateForTemplate(selectedTemplate, parts):
    # For each part in the selected template, pick the part with the lowest deformation cost
    newMesh = None
    obbMesh = None
    clustering = []
    doneCreation = False
    sampleChairBmp = "sample_chair/"
    if not exists(sampleChairBmp):
        makedirs(sampleChairBmp)
    max_iter = 5
    iterCreate = 0
    possibleMesh = []
    scores = []

    # open cluster file
    try:
        with open("clusterings", 'rb') as f:
            clustering = pickle.load(f)
            print('Sucessfully found clustering file')
        useCluster = True
    except:
        useCluster = False

    while (not doneCreation) and (iterCreate < max_iter):
        newMesh = None
        lastScore = 0.0
        templateParts = selectedTemplate.templateParts.copy()
        for templatePart in templateParts:
            # Select the part that has the obb that best fits the obb of the template part

            if useCluster == True:
                # find the closest cluster representative to the part and select a random cluster member
                selectedPart = None
                minDeformationCost = sys.float_info.max
                for cluster in clustering:
                    deformationCost = calculateDeformationCost(templatePart, cluster[0])
                    if deformationCost < minDeformationCost:
                        minDeformationCost = deformationCost
                        randomMember = random.randint(0,len(cluster)-1)
                        #print('len = ',len(cluster), ' rand = ',randomMember)
                        selectedPart = cluster[randomMember]
                    #print("cost: ",deformationCost, "mincost: ",minDeformationCost)


            else:
                # use a closest fitting part if there is no file for clusters
                selectedPart = None
                minDeformationCost = sys.float_info.max
                for part in parts:
                    if part.dir != templatePart.dir:
                        randN = random.uniform(0, 1)
                        deformationCost = calculateDeformationCost(templatePart, part) * randN
                        if deformationCost < minDeformationCost:
                            minDeformationCost = deformationCost
                            selectedPart = part
            selectedMesh = pymesh.load_mesh(chairsDirectory + "/" + selectedPart.dir + "/meshes/" + selectedPart.obj)

            # FOR DEBUGING: add the part to the obb mesh
            obbMesh = addToObbMesh(obbMesh, templatePart)

            # transform the selectedMesh so the OBB matches the templatePart's OBB
            selectedMesh = matchOBB(templatePart, selectedPart, selectedMesh)

            # add the selectedMesh to the newMesh
            newMesh = addToNewMesh(newMesh, selectedMesh)


            # check for symmetries and use that to fill in parts of the template
            for symmetry in templatePart.symmetries:
                symmetryPart = symmetry[0]
                symmetryAxes = symmetry[1]
                if symmetryPart in templateParts:
                    symmetricMesh = getSymmetric(templatePart, symmetryPart, symmetryAxes, selectedMesh)
                    newMesh = addToNewMesh(newMesh, symmetricMesh)

                    templateParts.remove(symmetryPart)

        # creates views and scores
        possibleMesh.append(newMesh)
        pymesh.save_mesh("sample_mesh.obj", newMesh);
        createViews.createViews("sample_mesh.obj", 1, sampleChairBmp)
        score = evaluate_sample.main(sampleChairBmp)
        scores.append(score[0])
        # print(scores)
        iterCreate += 1

        doneCreation = score[0] >= 0.8
        if iterCreate >= max_iter:
            index = scores.index(max(scores))
            newMesh = possibleMesh[index]

    pymesh.save_mesh("obbChair.obj", obbMesh);
    return newMesh

### Generate chairs

In [12]:
newBMPDir = 'new_chair_bmp/'
newChairDir = 'new_chair_obj/'
rankedChairDir = 'ranked_chair_obj/'
chairsDirectory = "clean_mesh"

print("Beginning Generation Script")
loadedTemplates = loadTemplates()
templates = loadedTemplates[0]
parts = loadedTemplates[1]
print("Templates Loaded")

chairCount = 1
for index, template in enumerate(templates):
    print("Generating Chair for: " + template.dir)
    newMesh = generateForTemplate(template, parts)
    print("New Chair Generated")

    file = newChairDir + str(index + 1).zfill(4) +  template.dir + ".obj"
    pymesh.save_mesh(file, newMesh);
    createViews.createViews(file, chairCount, newBMPDir)
    chairCount += 3

scores = evaluate_sample.main(newBMPDir)
fileList = listdir(newChairDir)
fileList.sort()
fileListBmp = listdir(newBMPDir)
for i in range(0, int(len(fileListBmp) / 3)):
    index, value = max(enumerate(scores), key=operator.itemgetter(1))
    scoredChair = fileList.pop(index)
    print(scoredChair)
    scores = np.delete(scores, index)
    mesh = pymesh.load_mesh(newChairDir + scoredChair)
    pymesh.save_mesh(rankedChairDir + str(i + 1) + ".obj", mesh)

Beginning Generation Script
Sucessfully found templates file
Templates Loaded
Generating Chair for: ChairSwivel01
Sucessfully found clustering file
['1.bmp', '2.bmp', '3.bmp']
1
2
3
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_device_fn': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ac95d4a20>, '_save_checkpoints_steps': None, '_model_dir': 'checkpoint/Top/', '_protocol': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_experimental_distribute': None, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 100, '_train_distribute': None, '_keep_checkpoint_max': 5, '_save_checkpoints_

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_device_fn': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ac9653400>, '_save_checkpoints_steps': None, '_model_dir': 'checkpoint/Front/', '_protocol': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_experimental_distribute': None, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 100, '_train_distribute': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_global_id_in_cluster': 0}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph w

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/Side/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[0.17605697]
done!
['1.bmp', '2.bmp', '3.bmp']
1
2
3
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_device_fn': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ac96d36d8>, '_save_checkpoints_steps': None, '_model_dir': 'checkpoint/Top/', '_protocol': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_experimental_distribute': None, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    met

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/Top/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_device_fn': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ac9122978>, '_save_checkpoints_steps': None, '_model_dir': 'checkpoint/Front/', '_protocol': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_experimental_distribute': None, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 100, '_train_distribute': None, '_keep_checkpoint_max': 5, '_sav

INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_device_fn': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ac9470a20>, '_save_checkpoints_steps': None, '_model_dir': 'checkpoint/Side/', '_protocol': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_experimental_distribute': None, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 100, '_train_distribute': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_global_id_in_cluster': 0}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/Side/model.ckpt-

...

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/Side/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[0.50533682]
done!
['1.bmp', '2.bmp', '3.bmp']
1
2
3
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_device_fn': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ac9578780>, '_save_checkpoints_steps': None, '_model_dir': 'checkpoint/Top/', '_protocol': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_experimental_distribute': None, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    met

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/Top/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_device_fn': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ac92ca748>, '_save_checkpoints_steps': None, '_model_dir': 'checkpoint/Front/', '_protocol': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_experimental_distribute': None, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 100, '_train_distribute': None, '_keep_checkpoint_max': 5, '_sav

INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_device_fn': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ac95fceb8>, '_save_checkpoints_steps': None, '_model_dir': 'checkpoint/Side/', '_protocol': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_experimental_distribute': None, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 100, '_train_distribute': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_global_id_in_cluster': 0}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/Side/model.ckpt-

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/Top/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_device_fn': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ac9289a20>, '_save_checkpoints_steps': None, '_model_dir': 'checkpoint/Front/', '_protocol': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_experimental_distribute': None, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/Front/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_service': None, '_device_fn': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1ac9527ef0>, '_save_checkpoints_steps': None, '_model_dir': 'checkpoint/Side/', '_protocol': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_experimental_distribute': None, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 100, '_train_distribute'